In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_json('messages.json')

In [3]:
data.columns

Index(['userId', 'exportDate', 'conversations'], dtype='object')

In [4]:
data['conversations'][0].keys()

dict_keys(['id', 'displayName', 'version', 'properties', 'threadProperties', 'MessageList'])

In [5]:
data['conversations'][0]['MessageList'][0].keys()

dict_keys(['id', 'displayName', 'originalarrivaltime', 'messagetype', 'version', 'content', 'conversationid', 'from', 'properties', 'amsreferences'])

In [6]:
#forming question and answer for each conversation
mod_data = []
for i in data['conversations']:
    mod_data.append([])
    for j in i['MessageList'][::-1]:
        msg = j['content']
        if j['messagetype']=='Event/Call' or '<' in msg:
                msg = 'Event/Special'
                
        if j['displayName']!=None:
            if not mod_data[-1]:
                mod_data[-1].append([msg,0])
            elif mod_data[-1][-1][1]==0:
                mod_data[-1][-1][0]+= ' '+msg
            else:
                mod_data[-1].append([msg,0])
        else:
            if not mod_data[-1]:
                mod_data[-1].append([msg,1])
            elif mod_data[-1][-1][1]==1:
                mod_data[-1][-1][0]+= ' '+msg
            else:
                mod_data[-1].append([msg,1])

In [7]:
temp1_data = []
for i in mod_data:
    for j in i:
        temp1_data.append(j[0].split())

In [8]:
#word2vec for each word
from nltk.tokenize import sent_tokenize, word_tokenize
import gensim
from gensim.models import Word2Vec

modelW2V = gensim.models.Word2Vec(temp1_data, min_count = 1,vector_size = 100, window = 5)

In [9]:
len(modelW2V.wv.key_to_index)

334120

In [10]:
#question and answer data for training
que_data = []
ans_data = []
for i in mod_data:
    t1 = []
    t2 = []
    for j in range(0, len(i)-1, 2):
        t1.append(i[j][0].split())
        t2.append(i[j+1][0].split())
    que_data.append(t1)
    ans_data.append(t2)

In [11]:
#Converting each word to vector
train_data = []
for i in que_data:
    t1 = []
    for j in i:
        t1.append(list(map(modelW2V.wv.get_vector, j)))
    train_data.append(t1)

In [12]:
#Converting each word to vector
test_data = []
for i in ans_data:
    t1 = []
    for j in i:
        t1.append(list(map(modelW2V.wv.get_vector, j)))
    test_data.append(t1)

In [15]:
#LSTM RNN using tensorflow
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model

class Autoencoder(Model):
    def __init__(self):
        super(Autoencoder, self).__init__()
        #self.latent_dim = 128
        self.encoder = tf.keras.Sequential([
          layers.Embedding(input_dim=100, output_dim=100),
          layers.LSTM(128, activation='relu', input_dim=100),
          layers.Dense(100, input_dim=128)
        ])
        self.decoder = tf.keras.Sequential([
          layers.Dense(100, activation='sigmoid'),
          layers.LSTM(128, activation='relu', input_dim=100),
          layers.Dense(100)
        ])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

autoencoder = Autoencoder()

In [ ]:
#training model (it was taking more time to train so not able to get results)
autoencoder.compile(tf.keras.optimizers.Adam(learning_rate=1e-3), loss=tf.keras.losses.BinaryCrossentropy())
autoencoder.fit(train_data, test_data)